## Import packages

In [ ]:
import functools

import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import numpy as np
import skill_metrics
import xarray as xr
from c3s_eqc_automatic_quality_control import diagnostics, download, plot, utils
from xarrayMannKendall import Mann_Kendall_test

plt.style.use("seaborn-v0_8-notebook")
plt.rcParams["hatch.linewidth"] = 0.5

## Define Parameters

In [ ]:
# Time
year_start = 1985
year_stop = 1995

# Define climatology years
clima_year_start = 1985
clima_year_stop = 1989
assert (clima_year_start >= year_start) and (clima_year_stop <= year_stop)

# Choose CORDEX or CMIP6
collection_id = "projections-cordex-domains-single-levels"  # "projections-cmip6"

# Define region for analysis
lon_slice = slice(-4, 20)
lat_slice = slice(35, 50)

# Define region for request
cordex_domain = "europe"

# Chunks for download
chunks = {"year": 1}
assert "month" not in chunks, "Do not use chunks smaller than 1y"

## Define models

In [ ]:
models_cordex = [
    "clmcom_clm_cclm4_8_17",
    "clmcom_eth_cosmo_crclim",
    "cnrm_aladin63",
    "dmi_hirham5",
    "knmi_racmo22e",
    "mohc_hadrem3_ga7_05",
    "mpi_csc_remo2009",
    "smhi_rca4",
    "uhoh_wrf361h",
]

models_cmip6 = [
    "access_cm2",
    "awi_esm_1_1_lr",
    "bcc_esm1",
    "cesm2_fv2",
    "cnrm_cm6_1",
    "fgoals_g3",
]

## Define ERA5 request

In [ ]:
request_era = (
    "reanalysis-era5-single-levels-monthly-means",
    {
        "product_type": "monthly_averaged_reanalysis",
        "format": "netcdf",
        "time": "00:00",
        "variable": "mean_total_precipitation_rate",
        "year": [str(year) for year in range(year_start - 1, year_stop + 1)],
        "month": [f"{month:02d}" for month in range(1, 12 + 1)],
    },
)

## Define model requests

In [ ]:
request_cordex = {
    "format": "zip",
    "domain": cordex_domain,
    "experiment": "historical",
    "horizontal_resolution": "0_11_degree_x_0_11_degree",
    "temporal_resolution": "monthly_mean",
    "variable": "mean_precipitation_flux",
    "gcm_model": "mpi_m_mpi_esm_lr",
    "ensemble_member": "r1i1p1",
}

request_cmip6 = {
    "format": "zip",
    "temporal_resolution": "monthly",
    "experiment": "historical",
    "variable": "precipitation",
    "year": [str(year) for year in range(year_start - 1, year_stop + 1)],
    "month": [f"{month:02d}" for month in range(1, 12 + 1)],
}


def get_cordex_years(
    year_start,
    year_stop,
    start_years=[1971, 1981, 1991, 2001],
    end_years=[1980, 1990, 2000, 2005],
):
    start_year = []
    end_year = []
    years = set(range(year_start - 1, year_stop + 1))
    for start, end in zip(start_years, end_years):
        if years & set(range(start, end + 1)):
            start_year.append(start)
            end_year.append(end)
    return start_year, end_year


if collection_id == "projections-cordex-domains-single-levels":
    weights = False  # Do not weight spatial statistics/errors
    periodic = False
    models = models_cordex
    model_key = "rcm_model"
    request_sim = (
        collection_id,
        [
            {
                **request_cordex,
                "start_year": start_year,
                "end_year": end_year,
            }
            for start_year, end_year in zip(*get_cordex_years(year_start, year_stop))
        ],
    )
elif collection_id == "projections-cmip6":
    weights = True  # Weight spatial statistics/errors
    periodic = True
    models = models_cmip6
    model_key = "model"
    request_sim = (
        "projections-cmip6",
        download.split_request(request_cmip6, chunks=chunks),
    )
else:
    raise ValueError(f"{collection_id=} is not supported for this notebook.")

## Functions to cache

In [ ]:
original_test = functools.partial(Mann_Kendall_test, alpha=0.05, method="theilslopes")


def get_annual_precipitation(ds, model, year_start, year_stop):
    # Select years (shift -1 to get D(year-1)J(year)F(year))
    ds = ds.assign_coords(year=ds["time"].dt.year.shift(time=-1))
    mask = (ds["year"] >= year_start) & (ds["year"] <= year_stop)
    ds = ds.where(mask.compute(), drop=True)
    ds["year"] = ds["year"].astype(int)

    # Varname
    varname = "mtpr" if model == "ERA5" else "pr"

    # Annual weighted mean
    da = (
        ds[varname]
        .groupby("year")
        .map(diagnostics.seasonal_weighted_mean)
        .stack(year_season=("year", "season"))
        .reset_index("year_season")
    )

    # Convert units
    with xr.set_options(keep_attrs=True):
        da = da * 3600 * 24
    da.attrs["units"] = "mm/day"

    return da


def compute_climatology(da, clima_year_start, clima_year_stop):
    mask = (da["year"] >= clima_year_start) & (da["year"] <= clima_year_stop)
    da = da.where(mask.compute(), drop=True)
    return da.mean("year_season", keep_attrs=True)


def compute_anomaly(da, clima_year_start, clima_year_stop):
    clima = compute_climatology(da, clima_year_start, clima_year_stop)
    anoma = (da - clima) * 100 / da
    anoma.attrs.update({"long_name": "precipitation anomaly", "units": "%"})
    return anoma


def spatial_weighted_trends(
    ds,
    model,
    year_start,
    year_stop,
    clima_year_start,
    clima_year_stop,
    weights,
    lon_slice=None,
    lat_slice=None,
):
    if lon_slice or lat_slice:
        ds = utils.regionalise(ds, lon_slice=lon_slice, lat_slice=lat_slice)
    if "precipitation" in ds:
        da = ds["precipitation"]
    else:
        da = diagnostics.spatial_weighted_mean(
            get_annual_precipitation(ds, model, year_start, year_stop),
            weights=True if model == "ERA5" else weights,
        )
    anoma = compute_anomaly(da, clima_year_start, clima_year_stop)

    # Compute anomaly trends
    ds = (
        original_test(
            anoma.expand_dims("x"),
            coords_name={"time": "year_season", "x": "x"},
        )
        .compute()
        .squeeze("x", drop=True)
    )

    # Add precipitation and anomaly
    ds["precipitation"] = da
    ds["precipitation_anomaly"] = anoma

    return ds.expand_dims(model=[model])


def regridded_trends(
    ds,
    model,
    year_start,
    year_stop,
    clima_year_start,
    clima_year_stop,
    grid_out=None,
    **kwargs,
):
    da = get_annual_precipitation(ds, model, year_start, year_stop)

    # Compute anomaly trends
    coords_name = {"time": "year_season"} | {
        k: v for k, v in zip(("x", "y"), ds[["longitude", "latitude"]].dims)
    }
    ds_trend = original_test(da, coords_name=coords_name).compute()
    ds_trend = ds_trend.rename({k: v for k, v in coords_name.items() if k != "time"})
    ds_trend = ds_trend.assign_coords(ds.drop_dims("time").coords)

    # Add precipitation and climatology
    clima = compute_climatology(da, clima_year_start, clima_year_stop)
    ds_trend["precipitation_climatology"] = clima
    if grid_out is not None:
        ds_trend = ds_trend.cf.add_bounds(
            (
                coord
                for coord in ("longitude", "latitude")
                if coord not in ds_trend.cf.bounds
            )
        )
        ds_trend = diagnostics.regrid(ds_trend, grid_out, **kwargs)
    return ds_trend.expand_dims(model=[model])

## Compute spatial weighted trends

In [ ]:
datasets = []
transform_func_kwargs = {
    "year_start": year_start,
    "year_stop": year_stop,
    "clima_year_start": clima_year_start,
    "clima_year_stop": clima_year_stop,
    "weights": weights,
}
for model in models + ["ERA5"]:
    print(f"Downloading and processing {model}")
    if model == "ERA5":
        request_model = request_era
    else:
        request_model = request_sim
        for request in request_model[1]:
            request[model_key] = model
    datasets.append(
        download.download_and_transform(
            *request_model,
            chunks=chunks if model == "ERA5" else {},
            transform_func=spatial_weighted_trends,
            transform_func_kwargs={
                "model": model,
                "lon_slice": lon_slice,
                "lat_slice": lat_slice,
                **transform_func_kwargs,
            },
            transform_chunks=False,
        )
    )

# Combine and add ensemble
ds_mean_trend = xr.concat(datasets, "model")
with xr.set_options(keep_attrs=True):
    ds_ensemble = ds_mean_trend.drop_sel(model="ERA5").mean("model")
ds_mean_trend = ds_mean_trend.merge(
    spatial_weighted_trends(ds_ensemble, model="ensemble", **transform_func_kwargs)
)

## Plot spatial weighted precipitation and anomaly

In [ ]:
for var in ["precipitation", "precipitation_anomaly"]:
    da = ds_mean_trend[var]
    da.drop_sel(model=["ensemble", "ERA5"]).plot(
        hue="model", linewidth=0.5, color="grey"
    )
    da.sel(model=["ensemble", "ERA5"]).plot(hue="model")
    plt.xticks(
        da["year_season"].values[::4] + 1 / 12, ds_mean_trend["year"].values[::4]
    )
    plt.xlabel("year")
    plt.grid()
    plt.show()

## Spatial weighted trends boxplot

In [ ]:
df_slope = (ds_mean_trend["trend"] * 10).to_dataframe()

ax = df_slope.boxplot()
ax.scatter(x=[1] * len(df_slope), y=df_slope, color="grey", marker=".")
for model in ["ensemble", "ERA5"]:
    ax.scatter(x=1, y=df_slope[df_slope.index == model], label=model, marker="o")
ax.set_ylabel("% / decade")
_ = plt.legend()

## Compute trend maps

In [ ]:
print("Downloading and processing ERA5")
transform_func_kwargs = {
    "year_start": year_start,
    "year_stop": year_stop,
    "clima_year_start": clima_year_start,
    "clima_year_stop": clima_year_stop,
    "method": "conservative",
}

ds_era = download.download_and_transform(
    *request_era,
    chunks=chunks,
    transform_func=regridded_trends,
    transform_func_kwargs={"model": "ERA5", **transform_func_kwargs},
    transform_chunks=False,
)

datasets = []
for model in models:
    print(f"Downloading and processing {model}")
    request_model = request_sim
    for request in request_model[1]:
        request[model_key] = model
    ds = download.download_and_transform(
        *request_model,
        transform_func=regridded_trends,
        transform_func_kwargs={
            "model": model,
            "grid_out": ds_era[["longitude", "latitude"]],
            "periodic": periodic,
            **transform_func_kwargs,
        },
        transform_chunks=False,
    )
    datasets.append(utils.regionalise(ds, lon_slice=lon_slice, lat_slice=lat_slice))
ds_era = utils.regionalise(ds_era, lon_slice=lon_slice, lat_slice=lat_slice)

ds_sim_regr = xr.concat(datasets, "model")
ds_ens = ds_sim_regr.mean("model").expand_dims(model=["ensemble"])
ds_all_regr = xr.concat([ds_era, ds_ens, ds_sim_regr], "model")

da_trend = ds_all_regr["trend"] * 1.0e3 / ds_all_regr["precipitation_climatology"]
da_trend.name = ""
da_trend.attrs.update({"units": "%/decade"})

## Define plotting kwargs

In [ ]:
Projection = (
    ccrs.Robinson
    if abs(lon_slice.stop - lon_slice.start) >= 360
    and abs(lat_slice.stop - lat_slice.start) >= 180
    else ccrs.PlateCarree
)
shading_kwargs = xr.plot.utils._determine_cmap_params(
    da_trend.values, levels=11, robust=True, cmap="bwr_r", extend="both"
)
shading_kwargs["projection"] = Projection(
    central_longitude=(lon_slice.stop + lon_slice.start) / 2
)
hatches_kwargs = {
    "plot_func": "contourf",
    "show_stats": False,
    "cmap": "none",
    "add_colorbar": False,
}

## Plot ERA5 trends

In [ ]:
model = "ERA5"
plot.projected_map(da_trend.sel(model=model), **shading_kwargs)
plot.projected_map(
    ds_all_regr["p"].sel(model=model),
    levels=[0, 0.05, 1],
    hatches=["", "/" * 4],
    **hatches_kwargs,
)
_ = plt.suptitle(f"Precipitation trend ({year_start}-{year_stop})")

## Plote esemble trends

In [ ]:
model = "ensemble"
is_signif = xr.where(
    ds_sim_regr["p"] <= 0.05,
    ds_sim_regr["trend"] / ds_ens["trend"].squeeze() > 0,
    False,
)
is_signif_ratio = is_signif.sum("model") / is_signif.sizes["model"]
is_signif_ratio = is_signif_ratio.expand_dims(model=[model])
plot.projected_map(da_trend.sel(model=model), stats_weights=weights, **shading_kwargs)
plot.projected_map(
    is_signif_ratio.squeeze(),
    levels=[0, 0.8, 1],
    hatches=["/" * 5, ""],
    **hatches_kwargs,
)
_ = plt.suptitle(f"Precipitation trend ({year_start}-{year_stop})")

## Plot precipitation trends for all models

In [ ]:
facet = plot.projected_map(
    da_trend.drop_sel(model=["ERA5", "ensemble"]),
    col="model",
    col_wrap=3,
    add_colorbar=False,
    **shading_kwargs,
)
for ax, sel in zip(facet.axs.flatten(), facet.name_dicts.flatten()):
    if not sel:
        continue
    da = ds_all_regr["p"].sel(**sel)
    plot.projected_map(
        da, ax=ax, levels=[0, 0.05, 1], hatches=["", "/" * 5], **hatches_kwargs
    )
plt.suptitle(f"Precipitation trend ({year_start}-{year_stop})")
cax = plt.axes([0.05, -0.04, 0.95, 0.04])
cbar1 = plt.colorbar(
    facet.axs[0][0].collections[0],
    cax=cax,
    orientation="horizontal",
    label=xr.plot.utils.label_from_attrs(da_trend),
)

## Plot precipitation trend bias for all models

In [ ]:
da_bias = (
    (ds_all_regr["trend"].drop_sel(model="ERA5") - ds_era["trend"].squeeze())
    * 10
    * 100
    / ds_era["precipitation_climatology"].squeeze()
)
da_bias.name = ""
da_bias.attrs.update({"units": "%/decade"})
facet = plot.projected_map(
    da_bias.drop_sel(model="ensemble"),
    col="model",
    col_wrap=3,
    add_colorbar=False,
    **shading_kwargs,
)
plt.suptitle(f"Precipitation trend bias ({year_start}-{year_stop})")
cax = plt.axes([0.05, -0.04, 0.95, 0.04])
cbar1 = plt.colorbar(
    facet.axs[0][0].collections[0],
    cax=cax,
    orientation="horizontal",
    label=xr.plot.utils.label_from_attrs(da_bias),
)

## Plot precipitation trend bias for ensamble

In [ ]:
model = "ensemble"
plot.projected_map(da_bias.sel(model=model), stats_weights=weights, **shading_kwargs)
_ = plt.suptitle(f"Precipitation trend bias ({year_start}-{year_stop})")

## Compute statistics

In [ ]:
ds_trend = da_trend.to_dataset(name="trend")
ds_stats = xr.concat(
    [
        diagnostics.spatial_weighted_statistics(
            ds_trend.drop_sel(model="ERA5"), weights=weights
        ),
        diagnostics.spatial_weighted_statistics(
            ds_trend.sel(model="ERA5"), weights=True
        ),
    ],
    "model",
)

ds_error = diagnostics.spatial_weighted_errors(
    ds_trend.drop_sel(model="ERA5"), ds_trend.sel(model="ERA5"), weights=weights
)
df_stats_and_error = xr.merge([ds_stats, ds_error])["trend"].to_pandas()
df_stats_and_error

## Taylor Diagram

In [ ]:
# TODO: needs tuning
skill_metrics.taylor_diagram(
    df_stats_and_error.loc["std"].values,
    df_stats_and_error.loc["crmse"].values,
    df_stats_and_error.loc["corr"].values,
    alpha=0.0,
    axismax=4,
    colCOR="k",
    colOBS="k",
    colRMS="m",
    colSTD="b",
    markerColor="r" if len(df_stats_and_error.columns) >= 9 else None,  # TODO
    markerLabel=list(df_stats_and_error.columns),
    markerLegend="on",
    markerSize=10,
    markerobs="o",
    styleCOR="--",
    styleOBS="--",
    styleRMS=":",
    styleSTD="-.",
    tickRMS=np.linspace(0, 4, 5),
    tickSTD=np.linspace(0, 4, 5),
    titleCOR="on",
    titleOBS="ERA5",
    titleRMS="on",
    titleRMSDangle=40.0,
    titleSTD="on",
    widthCOR=0.5,
    widthOBS=2,
    widthRMS=2,
    widthSTD=1.0,
)